# Phi-2
- Microsoft 가 발표한 Tiny LLM(???)
- 2.7B(27억) parameters
- https://huggingface.co/microsoft/phi-2

In [ ]:
### install (upgrade) huggingface transformers library.
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
# 노트북에 보기 좋게 출력하기 위한 Utillity
from IPython.display import display
from IPython.display import Markdown

import textwrap
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

torch.set_default_device("cuda")

- 몇몇 모델은 License 문제로 huggingface API key를 요구 할 수도 있으나 Phi-2 는 완전 공개 (MIT license) 모델이라 필요치 않음

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- Model card 의 예제는 greedy generation
- https://huggingface.co/docs/transformers/main/en/generation_strategies

In [ ]:
cfg = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    temperature=0.2,
    top_k=40,
    top_p=0.95,
    eos_token_id=model.config.eos_token_id
)

# 인풋 토큰 생성
inputs = tokenizer('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, generation_config=cfg)
text = tokenizer.batch_decode(outputs)[0]
to_markdown(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


> ```python
> def print_prime(n):
>    """
>    Print all primes between 1 and n
>    """
>    for i in range(2, n+1):
>        for j in range(2, i):
>            if i % j == 0:
>                break
>        else:
>            print(i)
> 
> print_prime(20)
> ```
> 
> In this example, we define a function `print_prime` that takes an integer `n` as input and prints all prime numbers between 1 and `n`. We use a nested loop to iterate over all numbers between 2 and `n`, and for each number, we check if it is divisible by any number between 2 and itself. If it is, we break out of the inner loop and move on to the next number. If it is not divisible by any number, we print it.
> 
> ## Exercises
> 
> 1. Write a function that takes a list of integers as input and returns the sum of all even numbers in the list.
> 
> ```python

In [ ]:
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       for j in range(2, i):
           if i % j == 0:
               break
       else:
           print(i)

print_prime(20)

2
3
5
7
11
13
17
19


- Local 에서 LLM 을 구동시키고 API 를 제공하기위한 패키지들이 존재함
  - https://github.com/abetlen/llama-cpp-python
  - https://github.com/vllm-project/vllm

- 모델을 더 작고 가볍게 만들어 낮은 성능의 PC에서도 실행 할 수 있게 하는 quantization 이라는 방법이 있음
  - https://huggingface.co/blog/4bit-transformers-bitsandbytes